In [1]:
import os
import sys
import math
import GPUtil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook as tqdm, tnrange
from transformers.tokenization_bert import BertTokenizer
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1203 03:25:13.443233 140284246263616 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import config
from fgc_preprocess import SerContextDataset, BertSpanIdx, bert_collate
from sup_model import BertSupSentClassification
from utils import read_fgc, read_hotpot

I1203 03:25:13.887823 140284246263616 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1203 03:25:14.890967 140284246263616 corenlp.py:118] The server is available.


In [4]:
items = read_fgc(config.FGC_DEV)

no gold supporting evidence
{'QID': 'D009Q03', 'QTYPE': '申论', 'QTEXT': '「占领华尔街」运动的诉求为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction'}
no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次签订的北美贸易协定从签署至生效过了几日?', 'ANSWER': [{'ATEXT': '资讯不足无法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}]}], 'ASPAN': [], 'SHINT': [0, 0, 0], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction'}
no gold supporting evidence
{'QID': 'D117Q05', 'QTYPE': '进阶题', 'QTEXT': '是否发现肿瘤就是得到癌症?', 'ANSWER': [{'ATEXT': '否', 'ATOKEN': [{'text': '否', 'st

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
dataset = SerContextDataset(items, transform=torchvision.transforms.Compose([BertSpanIdx(tokenizer)]))

I1203 03:27:13.897674 140284246263616 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [13]:
len(dataset[0]['input_ids'])

433